
# ChatGPT API Natural Language Magic: The Gathering Card Generator

by Max Woolf ([@minimaxir](https://twitter.com/minimaxir))

This Colab Notebook demonstrates how ChatGPT can be used to output structured JSON relatively consistently, which can be used for downstream applications such as using the data for template formatting.

This ChatGPT API sentiment analyzer requires an OpenAI account with a payment method attached to it/a free trial, and an [OpenAI API Key](https://platform.openai.com/account/api-keys). Running the setup cells by **mousing over the cells and pressing the Play button** will prompt you to input a key from that link and press Enter; it will not be saved to the Notebook.

If generated one at a time (e.g. `Create a Magic card`), each card takes ~280 ChatGPT tokens to generate, or ~$0.55 per 1,000 cards generated. If 5 cards are generated at a time (e.g. `Create five variations of Magic cards`), it takes ~123 tokens to generate a card on average, about half the cost, and therefore this Notebook will use that default.

## Setup

In [91]:
!pip install -q openai rich ujson

In [92]:
import openai
import os
import re
from rich.console import Console
import re
import getpass
import json
from jinja2 import Template
import sys
from tqdm.auto import tqdm
from google.colab import files
import ujson

api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


In [201]:
# card_extract_pattern = r'(\{"name":.*rarity.*\})'
card_extract_pattern = r'(\{.*\})'
def color_rarity(value, to_notebook=True):
    value = value.lower()
    # Can only display colors if printing to the notebook
    if to_notebook:
        if "mythic" in value:
            value = f"[bright_magenta]{value}[/bright_magenta]"
        elif "rare" in value:
            value = f"[bright_blue]{value}[/bright_blue]"
        elif "uncommon" in value:
            value = f"[bright_green]{value}[/bright_green]"
    return value

TEMPLATE = Template(
    """{{ c.name }}{% if c.manaCost %}  {{ c.manaCost }}{% endif %}
{{ c.type }}{% if c.text %}
{{ c.text }}{% endif %}{% if c.flavorText %}
{{ c.flavorText }}{% endif %}{% if c.pt %}
{{ c.pt }}{% elif c.loyalty %}
Loyalty: {{ c.loyalty }}{% endif %}
{{ c.rarity }}"""
)


def render_card(card_dict):
    return TEMPLATE.render(c=card_dict)

In [226]:
system = """You are an assistant who works as a Magic: The Gathering card designer. Create cards that are in the following card schema and JSON format. OUTPUT MUST FOLLOW THIS CARD SCHEMA AND JSON FORMAT. DO NOT EXPLAIN THE CARD. The output must also follow the Magic "color pie".

{"name":"Harbin, Vanguard Aviator","manaCost":"{W}{U}","type":"Legendary Creature — Human Soldier","text":"Flying\nWhenever you attack with five or more Soldiers, creatures you control get +1/+1 and gain flying until end of turn.","flavorText":"\"Yotia is my birthright, father. Let me fight for it.\"","pt":"3/2","rarity":"rare"}"""

def generate_magic_cards(prompt):
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        # stop="<|DONE|>",
        max_tokens=1500,  # sanity limit
        temperature=0.8,  # increasing temp higher than 0.8 may cause non-JSON output
    )
    # print(r["usage"])
    return r["choices"][0]["message"]["content"]


## Generate the Card!

Some example inputs:

- Create five variations of five-color Mythic Rare Artifact Magic cards.
- Create five variations of Magic cards with the text "You win the match"
- Create five variations of Planeswalker Magic cards with the subtype "Bob".
- Create five variations of Magic cards with "What's updog?" as the card flavor text.
- Create five variations of Magic cards named "Darth Vader" with a mana cost of atleast ten.
- Create five variations of multicolor Magic cards based on the War of 1812.
- Create five variations of Sorcery Magic cards based on Final Fantasy VII.
- Create ten variations of Magic cards based on a crossover between the War of 1812 and Final Fantasy VII.
- Create ten variations of Magic cards based on a crossover between Twitter and TikTok.

In addition to cost savings, the "five variations" framing ensures all generations are distinct.

All generations are saved to a text file in the Notebook sidebar, which can be downloaded locally.

`n_iterations` will rerun the same prompt _n_ amount of times: good if you want to generate a large batch of cards to a file.

In [242]:
from ujson import JSONDecodeError
from datetime import datetime
num_repeat_to_file = 10
output_width = 60


prompt = "Create ten variations of Magic cards based on a crossover between Twitter and TikTok." #@param {type:"string"}
to_file = False  # not quite working yet
n_iterations = 1 #@param {type:"slider", min:1, max:10, step:1}

console = Console(width=60, record=True)

for _ in tqdm(range(n_iterations)):
    success = True
    outputs = generate_magic_cards(prompt)

    cards = re.split(r"\n{2,}", outputs)
    processed_cards = []

    for card in cards:
        try:
            card = re.search(card_extract_pattern, card, re.S)
            if card is not None:
                card = str(card.group(0))
                card = card.replace('"",', '\"",')  # handle double quotes just in case
                card = ujson.loads(card)
                card["name"] = f"[bold]{card['name']}[/bold]"
                if "flavorText" in card:
                    card["flavorText"] = f"[italic]{card['flavorText']}[/italic]"
                card["rarity"] = color_rarity(card["rarity"])
                if "power" in card and "toughness" in card:
                    card["pt"] = f'{card["power"]}/{card["toughness"]}'
                processed_cards.append(render_card(card))
        except JSONDecodeError as e:
            console.print(f"[bright_red]<JSON Parsing Failed! 😭>\nCard output: {card}[/bright_red]", highlight=False)
            success = False
            continue
        except KeyError as e:
            console.print(f"[bright_red]<Missing a required card attribute! 😭>\nCard output: {card}[/bright_red]", highlight=False)
            success = False
            continue

    if len(processed_cards) == 0:
        success = False
        console.print(f"[bright_red]<No cards generated! 😭>\nChatGPT output: {outputs}[/bright_red]", highlight=False)

    if success:
        console.print(("\n" + "─" * output_width + "\n").join(processed_cards), highlight=False)
        console.print("─" * output_width, highlight=False)
        # console.print(f"ChatGPT Output:\n{outputs}", highlight=False)

console.save_text(f"cards_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

  0%|          | 0/1 [00:00<?, ?it/s]

TikTok Star  {1}{G}
Creature — Human Social Media Influencer
Whenever TikTok Star enters the battlefield, create a 1/1 
green Plant token.
{1}, {T}: Target creature gains trample until end of turn.
This influencer’s content is just too good to ignore!
2/2
common
────────────────────────────────────────────────────────────
Twitter Troll  {1}{R}
Creature — Human Troll
Whenever Twitter Troll attacks or blocks, you may discard a 
card. If you do, Twitter Troll gets +2/+0 and gains first 
strike until end of turn.
Don’t feed the trolls, just block them!
1/3
common
────────────────────────────────────────────────────────────
Hashtag Warrior  {2}{W}
Creature — Human Warrior
Vigilance, protection from black
{1}{W}, {T}: Put a +1/+1 counter on Hashtag Warrior. 
Activate this ability only if you control a creature with 
flying.
Fight for what you believe in, and add a hashtag while 
you're at it!
2/2
uncommon
────────────────────────────────────────────────────────────
TikTok Dance Challenge  {3}{U}
Sorcery
Tap any number of creatures you control. Then, create X 1/1 
blue Merfolk creature tokens, where X is the number of 
tapped creatures. Those tokens have flying until end of 
turn.
Let's see those dance moves!
uncommon
────────────────────────────────────────────────────────────
Twitter Outrage  {2}{B}{B}
Sorcery
Target opponent chooses a creature they control, then 
sacrifices it. If that creature was a Human, create two 1/1 
black Zombie creature tokens. If that creature was a 
non-Human, Twitter Outrage deals 4 damage to that creature's
controller.
The cancel culture strikes again!
rare
────────────────────────────────────────────────────────────
Follower's Frenzy  {3}{G}
Enchantment
Whenever a creature enters the battlefield under your 
control, put a +1/+1 counter on it.
Whenever you cast a creature spell with converted mana cost 
4 or greater, create two 1/1 green Elf creature tokens.
Likes and followers are nice, but what really counts are the
fans!
rare
────────────────────────────────────────────────────────────
Viral Video  {2}{R}
Instant
Target creature you control gets +3/+0 and gains trample and
haste until end of turn. If that creature deals combat 
damage to a player this turn, draw a card.
This will definitely go viral!
uncommon
────────────────────────────────────────────────────────────
Influencer's Insight  {1}{U}
Sorcery
Draw two cards, then discard a card unless you reveal a 
creature card from your hand.
Knowledge is power, and followers are currency.
common
────────────────────────────────────────────────────────────
Tweetstorm  {2}{U}{U}
Instant
Counter target spell unless its controller pays {X}, where X
is the number of creatures you control with power 2 or less.
When words fail, just start typing!
rare
────────────────────────────────────────────────────────────
TikTok Viral Challenge  {2}{R}
Enchantment
Whenever a creature enters the battlefield under your 
control, it gains haste until end of turn.
Whenever you cast a creature spell with converted mana cost 
2 or less, you may tap target creature an opponent controls.
That creature doesn't untap during its controller's next 
untap step.
It's time to take on the next challenge!
uncommon

────────────────────────────────────────────────────────────

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
